# BioSCape Data Access

The BioSCape data is stored in an S3 bucket associated with the SMCE environment. You can access this data in several ways:


## 1. Intake Catalog 

**For reflectance data only**

The simplest method of access is through the BioSCape intake catalog.

Make sure intake, intake-xarray, s3fs, zarr, rioxarray, jinja2 are installed in your conda environment.

In [2]:
import intake
import warnings
warnings.filterwarnings('ignore')

# Load the catalog
catalog = intake.open_catalog('s3://bioscape-data/bioscape_avng.yaml')
# access a specific reflectance file
data = catalog.ang20231022t092801.ang20231022t092801_001
data = data.read_chunked()

# write the crs using rioxarray
data.rio.write_crs(data.spatial_ref.attrs['crs_wkt'], inplace=True)
data

<xarray.Dataset> Size: 777MB
Dimensions:      (y: 607, wavelength: 425, x: 753)
Coordinates:
    spatial_ref  int64 8B 0
  * wavelength   (wavelength) float32 2kB 377.2 382.2 ... 2.496e+03 2.501e+03
  * x            (x) float64 6kB 2.909e+05 2.909e+05 ... 2.956e+05 2.956e+05
  * y            (y) float64 5kB 6.349e+06 6.349e+06 ... 6.345e+06 6.345e+06
Data variables:
    reflectance  (y, wavelength, x) float32 777MB dask.array<chunksize=(80, 425, 753), meta=np.ndarray>
Attributes: (12/19)
    description:          L2A Analytyical per-pixel surface retrieval
    samples:              753
    lines:                607
    bands:                425
    header offset:        0
    file type:            ENVI Standard
    ...                   ...
    band names:           ['channel_0', 'channel_1', 'channel_2', 'channel_3'...
    masked pixel noise:   2.7447400093078613
    ang pge input files:  ['bad_element_file=/scratch/achlus/airborne_sds/ang...
    ang pge run command:  ['/scratch/achlus/airborne_sds/ang_l1b_radiance/emi...
    bbl:                  ['0', '1', '1', '1', '1', '1', '1', '1', '1', '1', ...
    data ignore value:    -9999

In [3]:
%%time
# read the data into memory
data.reflectance.compute()

CPU times: user 2 s, sys: 1.5 s, total: 3.5 s
Wall time: 6.67 s


<xarray.DataArray 'reflectance' (y: 607, wavelength: 425, x: 753)> Size: 777MB
array([[[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],

       [[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],

       [[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
...
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],

       [[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],

       [[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]]],
      dtype=float32)
Coordinates:
    spatial_ref  int64 8B 0
  * wavelength   (wavelength) float32 2kB 377.2 382.2 ... 2.496e+03 2.501e+03
  * x            (x) float64 6kB 2.909e+05 2.909e+05 ... 2.956e+05 2.956e+05
  * y            (y) float64 5kB 6.349e+06 6.349e+06 ... 6.345e+06 6.345e+06

## 2. Cropping API Access

**For reflectance data only**

An API is available that allows you to:

1. **Submit a GeoJSON**: This request returns the overlapping flightlines.
2. **Retrieve Data Cropped**: This request returns cropped data in NetCDF fromat. Provide a flightline, subsection number, a geojson, and an output file name.

The API can be used to download cropped data to any machine. It does require that you have a BioSCape SMCE username and password.

In [1]:
import requests
import json
import random
import os

URLTOKEN = "https://crop.bioscape.io/token/"
URLCROP = "https://crop.bioscape.io/crop/"
URLOVERLAP = "https://crop.bioscape.io/overlap/"

1. Set your variables

In [2]:
GEOJSON_FILE = "path_to_your_geojson"
OUTPUT_FILE = "output_file_name.nc" #This must be a netcdf!!!
SMCE_USERNAME = 'your_smce_username'
SMCE_PASSWORD = 'your_smce_password'

2. Get an Access Token

In [4]:
response = requests.post(
    URLTOKEN,
    data={"username": SMCE_USERNAME, "password": SMCE_PASSWORD},
    headers={"Content-Type": "application/x-www-form-urlencoded"}
)
response.raise_for_status()  # Raise an error for bad HTTP status codes
access_token = response.json().get('access_token')
response

<Response [200]>

3. Find out which files overlap with your GeoJson

In [5]:
with open(GEOJSON_FILE, 'rb') as f:
    response = requests.post(
        URLOVERLAP,
        headers={"Authorization": f"Bearer {access_token}"},
        files={"geojson": f}
    )
response.raise_for_status()  # Raise an error for bad HTTP status codes

# Extract the response body and HTTP status code
response_body = response.json()
files = response_body.get('files', [])
files

['ang20231022t092801_000',
 'ang20231022t094938_035',
 'ang20231022t094938_036',
 'ang20231029t120919_045',
 'ang20231029t123011_001',
 'ang20231029t123011_002']

4. Select which file you want to crop

In [6]:
flightline = 'ang20231022t094938'
subsection = 35

5. Request the cropped data

In [10]:
with open(GEOJSON_FILE, 'rb') as f:
    response = requests.post(
        URLCROP,
        headers={"Authorization": f"Bearer {access_token}", "Accept": "application/x-netcdf"},
        files={
            "geojson": f,
            "flightline": (None, flightline),
            "subsection": (None, str(subsection)),
            "outpath": (None, OUTPUT_FILE)
        }
    )
response.raise_for_status() # Raise an error for bad HTTP status code
response

<Response [200]>

6. Access the data with Xarray or write the output

In [8]:
import xarray as xr
import io

# Make sure you have the appropriate packages installed to use Xarary's netCDF backend
import h5netcdf

# Read the NetCDF data into an xarray Datase
dataset = xr.open_dataset(io.BytesIO(response.content))
dataset

<xarray.Dataset> Size: 87MB
Dimensions:      (wavelength: 425, x: 202, y: 253)
Coordinates:
  * wavelength   (wavelength) float32 2kB 377.2 382.2 ... 2.496e+03 2.501e+03
  * x            (x) float64 2kB 2.927e+05 2.927e+05 ... 2.939e+05 2.939e+05
  * y            (y) float64 2kB 6.352e+06 6.352e+06 ... 6.35e+06 6.35e+06
Data variables:
    spatial_ref  int64 8B ...
    reflectance  (y, wavelength, x) float32 87MB ...

In [9]:
# Write the response content to the output file
with open(OUTPUT_FILE, 'wb') as f:
    f.write(response.content)

## 3. S3FS and Rioxarray

For smaller files, you can use s3fs and rioxarray to access the data. For larger files it can take too long to read the data in from S3

In [3]:
import rioxarray as rxr
import os
import s3fs

In [4]:
s3 = s3fs.S3FileSystem(anon=False)
files = s3.ls('bioscape-data/')
files

['bioscape-data/AVNG',
 'bioscape-data/LVIS',
 'bioscape-data/PRISM',
 'bioscape-data/bioscape_avng.yaml',
 'bioscape-data/old']

In [5]:
sub_files = s3.ls('bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000')
sub_files

['bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000/ang20231022t092801_000_L1B_ORT_main_46dd9a4a_LOC',
 'bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000/ang20231022t092801_000_L1B_ORT_main_46dd9a4a_LOC.hdr',
 'bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000/ang20231022t092801_000_L1B_ORT_main_46dd9a4a_LOC_ORT',
 'bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000/ang20231022t092801_000_L1B_ORT_main_46dd9a4a_LOC_ORT.hdr',
 'bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000/ang20231022t092801_000_L1B_ORT_main_46dd9a4a_OBS',
 'bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000/ang20231022t092801_000_L1B_ORT_main_46dd9a4a_OBS.hdr',
 'bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000/ang20231022t092801_000_L1B_ORT_main_46dd9a4a_OBS_ORT',
 'bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000/ang20231022t092801_000_L1B_ORT_main_46dd9a4a_OBS_ORT.hdr',
 'bioscape-data/AVNG/ang20231022t092801/ang20231022t0928

In [6]:
%%time
print(f"File Name: {sub_files[4]}")
rxr.open_rasterio(os.path.join('s3://', sub_files[4])).compute()

File Name: bioscape-data/AVNG/ang20231022t092801/ang20231022t092801_000/ang20231022t092801_000_L1B_ORT_main_46dd9a4a_OBS
CPU times: user 136 ms, sys: 74.2 ms, total: 210 ms
Wall time: 2.81 s


<xarray.DataArray (band: 11, y: 640, x: 598)> Size: 34MB
array([[[6.77071207e+03, 6.76996725e+03, 6.76925446e+03, ...,
         6.82967919e+03, 6.83110790e+03, 6.83256256e+03],
        [6.76923141e+03, 6.76845597e+03, 6.76826575e+03, ...,
         6.82942614e+03, 6.83086338e+03, 6.83231042e+03],
        [6.76780447e+03, 6.76698891e+03, 6.76702999e+03, ...,
         6.82982604e+03, 6.83061646e+03, 6.83205588e+03],
        ...,
        [6.94256925e+03, 6.94002944e+03, 6.93750278e+03, ...,
         6.83597042e+03, 6.83751856e+03, 6.83906922e+03],
        [6.94251524e+03, 6.93997801e+03, 6.93745112e+03, ...,
         6.83617078e+03, 6.83770733e+03, 6.83925169e+03],
        [6.94246873e+03, 6.93993107e+03, 6.93740376e+03, ...,
         6.83630924e+03, 6.83786172e+03, 6.83942239e+03]],

       [[7.57674757e+01, 7.57518479e+01, 7.57360533e+01, ...,
         2.77185195e+02, 2.77170443e+02, 2.77155902e+02],
        [7.57676990e+01, 7.57520738e+01, 7.57362818e+01, ...,
         2.77182371e+02, 2.77167627e+02, 2.77153095e+02],
        [7.57679225e+01, 7.57522998e+01, 7.57365103e+01, ...,
         2.77179547e+02, 2.77164811e+02, 2.77150287e+02],
...
        [9.47583334e+00, 9.47583334e+00, 9.47583334e+00, ...,
         9.47583334e+00, 9.47583334e+00, 9.47583334e+00],
        [9.47583334e+00, 9.47583334e+00, 9.47583334e+00, ...,
         9.47583334e+00, 9.47583334e+00, 9.47583334e+00],
        [9.47583334e+00, 9.47583334e+00, 9.47583334e+00, ...,
         9.47583334e+00, 9.47583334e+00, 9.47583334e+00]],

       [[9.95220000e-01, 9.95220000e-01, 9.95220000e-01, ...,
         9.95220000e-01, 9.95220000e-01, 9.95220000e-01],
        [9.95220000e-01, 9.95220000e-01, 9.95220000e-01, ...,
         9.95220000e-01, 9.95220000e-01, 9.95220000e-01],
        [9.95220000e-01, 9.95220000e-01, 9.95220000e-01, ...,
         9.95220000e-01, 9.95220000e-01, 9.95220000e-01],
        ...,
        [9.95220000e-01, 9.95220000e-01, 9.95220000e-01, ...,
         9.95220000e-01, 9.95220000e-01, 9.95220000e-01],
        [9.95220000e-01, 9.95220000e-01, 9.95220000e-01, ...,
         9.95220000e-01, 9.95220000e-01, 9.95220000e-01],
        [9.95220000e-01, 9.95220000e-01, 9.95220000e-01, ...,
         9.95220000e-01, 9.95220000e-01, 9.95220000e-01]]])
Coordinates:
  * band         (band) int64 88B 1 2 3 4 5 6 7 8 9 10 11
  * x            (x) float64 5kB 0.5 1.5 2.5 3.5 4.5 ... 594.5 595.5 596.5 597.5
  * y            (y) float64 5kB 0.5 1.5 2.5 3.5 4.5 ... 636.5 637.5 638.5 639.5
    spatial_ref  int64 8B 0
Attributes: (12/26)
    Band_1:          Path length (m)
    Band_2:          To-sensor azimuth (0 to 360 degrees cw from N)
    Band_3:          To-sensor zenith (0 to 90 degrees from zenith)
    Band_4:          To-sun azimuth (0 to 360 degrees cw from N)
    Band_5:          To-sun zenith (0 to 90 degrees from zenith)
    Band_6:          Solar phase
    ...              ...
    header_offset:   0
    interleave:      bip
    lines:           640
    samples:         598
    line_averaging:  3
    pixel_size_(m):  6.3

## 4. Direct Download

Data is available for direct download from the [JPL endpoint](https://popo.jpl.nasa.gov/avng/).

Additionally, Kit Lewers has developed a script for easy data extraction from the JPL BioSCape endpoint. Her code can be found [here](https://github.com/kllewers/BioSCrapes).